In [6]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# tensorflow data setten %60lık eğitim verisini alır
# eğitim verisinin kalan %40lık kısmını doğrulama için kullanır
# test verisinin tamamını getirir
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

train_size = tf.data.experimental.cardinality(train_data).numpy()
validation_size = tf.data.experimental.cardinality(validation_data).numpy()
test_size = tf.data.experimental.cardinality(test_data).numpy()

# İndirilen eğitim, doğrulama ve test verisi miktarları yazdırılır.
print(f"Training data size: {train_size}")
print(f"Validation data size: {validation_size}")
print(f"Test data size: {test_size}")

#İndirilen eğitim verisi etiket ve cümle şeklindedir. Etiket olumsuz yorumlar için 0 olumlu yorumlar için 1 olacak şekildedir.
#Bunu görmek için indirilen eğitim verilerinden ilk 5 tanesinin cümle ve etiket değerlerini yazdıralım
print("----Eğitim Verilerinden Örnekler-----\n")
for i, (review, label) in enumerate(train_data.take(5)):
    print(f"Veri {i+1}:")
    print(f"Etiket: {'Olumlu' if label == 1 else 'Olumsuz'}")  # Etiketin pozitif veya negatif olduğunu göster
    print(f"Yorum: {review.numpy().decode('utf-8')}")  # Review'u numpy dizisinden string'e çevirme
    print("\n")

#imdb yorumlarını olumlu ve olumsuz olarak ayıracağız. Yani metinler olumlu ya da olumsuz şeklinde ayrılacak.
#Bunun için metinleri gömme vektörlerine dönüştürmeliyiz.
#tensorflowdan gömme için önceden eğitilmiş google/nnlm-en-dim50/2 isimli metin gömme modelini kullanacağız
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)

#tensorflow keras kullanılarak model oluşturulur ve model özeti yazdırılır
print("Model :\n")
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

#model optimize edici ve kayıp fonksiyonu kullanalım
#binary_crossentropy kayıp fonksiyonunu kullanacağız. Kayıp fonksiyonu için farklı seçeneklerimiz de var
#ancak takip ettiğimiz tutorial olasılıklara başa çıkmak için bu fonksyionun daha iyi olduğunu söylüyor.
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

#modeli 512'li gruplar halinde 10 dönem için eğitiyoruz
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

#test verisi ile modeli test ediyoruz
results = model.evaluate(test_data.batch(512), verbose=2)

#Modelin test sonucunu ekrana yazdırıyoruz
print("Model Test Sonuç Değerleri\n")
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

# test için kullandığımız veri setinden rastgele 10 tanesini alalım
test_data_random = test_data.shuffle(10000).take(10)

#Şimdi test veri setinden rastgele aldığımız 10 kaydı ve modelin o kayıtlar için yaptığı tahmini yazdırarak modelin başarısını analiz edelim
for i, (review, label) in enumerate(test_data_random):
    # Modelin tahminini hesapla
    prediction = model.evaluate(tf.expand_dims(review, 0))
    predicted_label = 'Olumlu' if prediction[0] >= 0.5 else 'Olumsuz'

    # Sonuçları yazdırma
    print(f"Test {i + 1}:")
    print(f"Yorum: {review.numpy().decode('utf-8')}")
    print(f"Gerçek Etiket: {'Olumlu' if label == 1 else 'Olumsuz'}")
    print(f"Tahmin Etiket : {predicted_label}")  # Tahmin edilen etiket
    print("\n")


Training data size: 15000
Validation data size: 10000
Test data size: 25000
----Eğitim Verilerinden Örnekler-----

Veri 1:
Etiket: Olumsuz
Yorum: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.


Veri 2:
Etiket: Olumsuz
Yorum: I have been known to fall asleep during films, but this is usually due to a combination of things including, re